## Aufgabe 2.3 - Schadstoffreduzierung eines Stahlwerks

In [1]:
import pyomo.environ as pyo


a)

Die Zielfunktion für das Optimierungsproblem ergibt sich zu: <br>

$$ min \sum A_{j,i}*x_{i}*c{i} $$ <br>

wobei $A$ die Matrix für die möglich Emmisionsreduktion pro Technologie und Hochofen dar stellt, $x$ der prozentuelle Einsatz der Technologie ist, um $c$ die jeweiligen Kosten für die Anwendung einer Technologie.
<br>

$A$ in 1000 Tonnen:

| Schadstoff |**Schornsteine 1**|**Schornsteine 2** |**Filter 1**|**Filter 2**|**Brennstoffe 1**|**Brennstoffe 2** |
|---         |   ---            |  ---              |   ---      |   ---      |   ---           |   ---            |
|Staub Ruß   |       12         |         9         |     25     |     20     |        17       |       13         |
|Schwefeloxid|       35         |         42        |     18     |     31     |        56       |       49         |
|Kohlenwasserstoffe|    37      |         53        |     28     |     24     |        29       |       30         |
<br>

$c$ in Mio. EUR:
<br>

||**Schornsteine 1**|**Schornsteine 2** |**Filter 1**|**Filter 2**|**Brennstoffe 1**|**Brennstoffe 2** |
|---|   ---            |  ---              |   ---      |   ---      |   ---           |   ---            |
|Kosten|       8          |         10        |     7      |     6      |        11       |       9          |
<br>

$x$ in %:

||**Schornsteine 1**|**Schornsteine 2** |**Filter 1**|**Filter 2**|**Brennstoffe 1**|**Brennstoffe 2** |
|---|   ---            |  ---              |   ---      |   ---      |   ---           |   ---            |
|Prozent|       ?          |         ?        |     ?      |     ?      |        ?       |       ?          |
<br>

Die Nebenbedingung ist:
$$ b <= A_{j,i} * x_{i}  $$

mit $b$ als Paramter der Schadstoffe die mindestens Eingespart werden müssen (in 1000 Tonnen):

||**Staub Russ**|**Schwefeloxid** |**Kohlenwasserstoffe**|
|---|   ---            |  ---              |   ---    |  
| |       60          |         150        |     125      |  
<br>


Im folgenden wird der Code für das Model bereit gestellt und die Lösung für a), b) und c) dargestellt.

In [2]:
# Aufbereiten der Modelleingangsparameter:
A = [[12,9,25,20,17,13],
     [35,42,18,31,56,49],
     [37,53,28,24,29,20]]
b = [60,150,125]
c = [8, 10,7,6,11,9]

Schadstoffe = ["staub_russ", "schwefel", "CH"]

Masnahmen = ["Schornstein_1", "Schornstein_2", 
             "Filter_1", "Filter_2", 
             "Brennstoff_1", "Brennstoff_2"]


dictionary = {}
reduktion = {}
cost = {}
for reihe, stoff in enumerate(Schadstoffe):
    reduktion[stoff] = b[reihe]
    for spalte, mas in enumerate(Masnahmen):       
        dictionary[(stoff, mas)] = A[reihe][spalte]

for i, mas in enumerate(Masnahmen):
    cost[mas] = c[i]   

In [20]:
# erstellen des Models + Lösung b):

model = pyo.AbstractModel()

model.schadstoffe = pyo.Set(initialize=Schadstoffe)
model.masnahmen = pyo.Set(initialize=Masnahmen)


model.A = pyo.Param(model.schadstoffe, model.masnahmen, initialize=dictionary)
model.b = pyo.Param(model.schadstoffe, initialize=reduktion)
model.c = pyo.Param(model.masnahmen, initialize=cost)

model.x = pyo.Var(model.masnahmen, bounds=(0,1), within=pyo.NonNegativeReals)


def zielfunktion(model):   
    return sum(model.A[j,i] * model.x[i] * model.c[i] for j in model.schadstoffe  for i in model.masnahmen)
model.cost = pyo.Objective(rule=zielfunktion, sense=pyo.minimize)


def schadstoff_rule(model, j):
        value = sum(model.A[j,i] * model.x[i] for i in model.masnahmen)
        return model.b[j] <= value
model.Nebenbedingung = pyo.Constraint(model.schadstoffe, rule=schadstoff_rule)


instance = model.create_instance()
opt = pyo.SolverFactory("gurobi")
results = opt.solve(instance)

instance.display()

x  = [instance.x[t]() for t in instance.masnahmen]

           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set schadstoffe; 1 index total
           0 seconds to construct Set schadstoffe; 1 index total
           0 seconds to construct Set schadstoffe; 1 index total
           0 seconds to construct Set masnahmen; 1 index total
           0 seconds to construct Set masnahmen; 1 index total
           0 seconds to construct Set masnahmen; 1 index total
           0 seconds to construct Set A_index; 1 index total
           0 seconds to construct Set A_in

In [24]:
# erstellen des Models + Lösung c):

model = pyo.AbstractModel()

model.schadstoffe = pyo.Set(initialize=Schadstoffe)
model.masnahmen = pyo.Set(initialize=Masnahmen)


model.A = pyo.Param(model.schadstoffe, model.masnahmen, initialize=dictionary)
model.b = pyo.Param(model.schadstoffe, initialize=reduktion)
model.c = pyo.Param(model.masnahmen, initialize=cost)

model.x = pyo.Var(model.masnahmen, bounds=(0,1), within=pyo.Binary)


def zielfunktion(model):   
    return sum(model.A[j,i] * model.x[i] * model.c[i] for j in model.schadstoffe  for i in model.masnahmen)
model.cost = pyo.Objective(rule=zielfunktion, sense=pyo.minimize)


def schadstoff_rule(model, j):
        value = sum(model.A[j,i] * model.x[i] for i in model.masnahmen)
        return model.b[j] <= value
model.Nebenbedingung = pyo.Constraint(model.schadstoffe, rule=schadstoff_rule)


instance_2 = model.create_instance()
opt = pyo.SolverFactory("gurobi")
results = opt.solve(instance_2)

instance_2.display()

x_2  = [instance.x[t]() for t in instance.masnahmen]

           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set schadstoffe; 1 index total
           0 seconds to construct Set schadstoffe; 1 index total
           0 seconds to construct Set schadstoffe; 1 index total
           0 seconds to construct Set masnahmen; 1 index total
           0 seconds to construct Set masnahmen; 1 index total
           0 seconds to construct Set masnahmen; 1 index total
           0 seconds to construct Set A_index; 1 index total
           0 seconds to construct Set A_in

Damit ergeben sich für Aufgabe 2.3.3 b) und c):

b)
$x_b$ in %:

||**Schornsteine 1**|**Schornsteine 2** |**Filter 1**|**Filter 2**|**Brennstoffe 1**|**Brennstoffe 2** |
|---|   ---            |  ---              |   ---      |   ---      |   ---           |   ---            |
|Prozent|      100        |       62.27       |     34.35    |    100    |        4.75    |       100        |
<br>

c)
$x_c$ in %:

||**Schornsteine 1**|**Schornsteine 2** |**Filter 1**|**Filter 2**|**Brennstoffe 1**|**Brennstoffe 2** |
|---|   ---            |  ---              |   ---      |   ---      |   ---           |   ---            |
|Prozent|      100        |       100       |     100    |    0    |        100    |      0       |
<br>

d) Wenn nur ein Typ für jede Maßnahme eingesetzt werden kann ist das Problem unlösbar, da die Nebenbedingung (erforderlichen Reduktionen) nicht erreicht werden kann:

In [32]:
# erstellen des Models + Lösung d):

model = pyo.AbstractModel()

model.schadstoffe = pyo.Set(initialize=Schadstoffe)
model.masnahmen = pyo.Set(initialize=Masnahmen)


model.A = pyo.Param(model.schadstoffe, model.masnahmen, initialize=dictionary)
model.b = pyo.Param(model.schadstoffe, initialize=reduktion)
model.c = pyo.Param(model.masnahmen, initialize=cost)

model.x = pyo.Var(model.masnahmen, bounds=(0,1), within=pyo.Binary)


def zielfunktion(model):   
    return sum(model.A[j,i] * model.x[i] * model.c[i] for j in model.schadstoffe  for i in model.masnahmen)
model.cost = pyo.Objective(rule=zielfunktion, sense=pyo.minimize)


def schadstoff_rule(model, j):
        value = sum(model.A[j,i] * model.x[i] for i in model.masnahmen)
        return model.b[j] <= value
model.Nebenbedingung = pyo.Constraint(model.schadstoffe, rule=schadstoff_rule)

def extra_nebenbedingung(model):
    for i in [2,4,6]:
        return x[i] + x[i-1] <= 1 #, x[3] + x[4] <=1, x[5] + x[6]<=1 
model.extra_NB = pyo.Constraint(rule=extra_nebenbedingung)
#"Schornstein_1", "Schornstein_2", 
 #            "Filter_1", "Filter_2", 
  #           "Brennstoff_1", "Brennstoff_2"]

instance_2 = model.create_instance()
opt = pyo.SolverFactory("gurobi")
results = opt.solve(instance_2)

instance_2.display()

x_2  = [instance.x[t]() for t in instance.masnahmen]

           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set Any; 1 index total
           0 seconds to construct Set schadstoffe; 1 index total
           0 seconds to construct Set schadstoffe; 1 index total
           0 seconds to construct Set schadstoffe; 1 index total
           0 seconds to construct Set masnahmen; 1 index total
           0 seconds to construct Set masnahmen; 1 index total
           0 seconds to construct Set masnahmen; 1 index total
           0 seconds to construct Set A_index; 1 index total
           0 seconds to construct Set A_in

ValueError: Invalid constraint expression. The constraint expression resolved to a trivial Boolean (True) instead of a Pyomo object. Please modify your rule to return Constraint.Feasible instead of True.

Error thrown for Constraint 'extra_NB'